# Workshop de Python

Cargamos datasets con información de admisiones a hospitales de enfermos de diabetes. El objetivo es, una vez limpiado el dataset, estudiarlo para extraer el máximo número de insights de los datos.

## Cargar librerías

In [22]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling

## Lectura de datos
- diabetic_data.csv
- admission_source_id.csv
- admission_type_id.csv
- discharge_disposition_id.csv

In [23]:
admin_src_df = pd.read_csv('data/admission_source_id.csv', 
                           na_values=["Not Available","None","Unknown/Invalid","Not Mapped"])
admin_type_df = pd.read_csv('data/admission_type_id.csv',
                           na_values = ["Not Available","None","Not Mapped"])
diabetic_df = pd.read_csv('data/diabetic_data.csv',
                          na_values=["Not Available","None","Unknown/Invalid","Not Mapped","?"])
dschrge_disp_df = pd.read_csv('data/discharge_disposition_id.csv',
                             na_values=["Not Available","None","Unknown/Invalid","Not Mapped"])
#id_mapping_df = pd.read_csv('data/IDs_mapping.csv')    #Es la colección de todas las demás tabals de referencia

C:\Users\c804324\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
diabetic_df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [19]:
pandas_profiling.ProfileReport(diabetic_df)

Number of variables,50
Number of observations,101766
Total Missing (%),7.4%
Total size in memory,38.8 MiB
Average record size in memory,400.0 B
Numeric,13
Categorical,35
Date,0
Text (Unique),0
Rejected,2
Distinct count,4


### comprobar correcta lectura de los datos
Revisemos qué hay en cada uno de los dataframes

In [25]:
#admin_src_df
#admin_type_df
dschrge_disp_df
#diabetic_df.head()

,discharge_disposition_id,description
0,1,Discharged to home
1,2,Discharged/transferred to another short term h...
2,3,Discharged/transferred to SNF
3,4,Discharged/transferred to ICF
4,5,Discharged/transferred to another type of inpa...
5,6,Discharged/transferred to home with home healt...
6,7,Left AMA
7,8,Discharged/transferred to home under care of H...
8,9,Admitted as an inpatient to this hospital
9,10,Neonate discharged to another hospital for neo...


Asignemos los índices originales a los ficheros de las tabals de referencia para desnormalizar la tabla _diabetic_ _ _df_

In [26]:
admin_src_df.set_index("admission_source_id", inplace = True)
admin_type_df.set_index("admission_type_id", inplace = True)
dschrge_disp_df.set_index("discharge_disposition_id", inplace = True)

Cambiemos el nombre de las descripciones de las tablas de referencia

In [27]:
admin_src_df.rename(columns={"description":"admin_src_desc"}, inplace = True)
admin_type_df.rename(columns={"description":"admin_type_desc"}, inplace = True)
dschrge_disp_df.rename(columns={"description":"dschrge_disp_desc"}, inplace = True)

### juntar todos los datos en el mismo DataFrame

Ahora sí ejecutemos los joins. Verifiquemos que si las relaciones son en efecto uno a uno

In [28]:
print(diabetic_df.join(admin_src_df,how="left", on = "admission_source_id").shape)
print(diabetic_df.join(admin_type_df,how="left", on = "admission_type_id").shape)
print(diabetic_df.join(dschrge_disp_df,how="left", on = "discharge_disposition_id").shape)
print(diabetic_df.shape)

(101766, 51)
(101766, 51)
(101766, 51)
(101766, 50)


In [29]:
aug_diabetic_df = diabetic_df.join(admin_src_df,how="left", on="admission_source_id")
aug_diabetic_df = aug_diabetic_df.join(admin_type_df,how="left", on = "admission_type_id")
aug_diabetic_df = aug_diabetic_df.join(dschrge_disp_df,how="left", on = "discharge_disposition_id")

Quitemos los id's que ya denormalizamos

In [30]:
aug_diabetic_df.drop(labels = ["admission_type_id","discharge_disposition_id","admission_source_id"], 
                     axis = 1, 
                     inplace = True)

In [31]:
aug_diabetic_df

,encounter_id,patient_nbr,race,gender,age,weight,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admin_src_desc,admin_type_desc,dschrge_disp_desc
0,2278392,8222157,Caucasian,Female,[0-10),NaN,1,NaN,Pediatrics-Endocrinology,41,...,No,No,No,No,No,No,NO,Physician Referral,NaN,NaN
1,149190,55629189,Caucasian,Female,[10-20),NaN,3,NaN,NaN,59,...,No,No,No,No,Ch,Yes,>30,Emergency Room,Emergency,Discharged to home
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,2,NaN,NaN,11,...,No,No,No,No,No,Yes,NO,Emergency Room,Emergency,Discharged to home
3,500364,82442376,Caucasian,Male,[30-40),NaN,2,NaN,NaN,44,...,No,No,No,No,Ch,Yes,NO,Emergency Room,Emergency,Discharged to home
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,NaN,NaN,51,...,No,No,No,No,Ch,Yes,NO,Emergency Room,Emergency,Discharged to home
5,35754,82637451,Caucasian,Male,[50-60),NaN,3,NaN,NaN,31,...,No,No,No,No,No,Yes,>30,Clinic Referral,Urgent,Discharged to home
6,55842,84259809,Caucasian,Male,[60-70),NaN,4,NaN,NaN,70,...,No,No,No,No,Ch,Yes,NO,Clinic Referral,Elective,Discharged to home
7,63768,114882984,Caucasian,Male,[70-80),NaN,5,NaN,NaN,73,...,No,No,No,No,No,Yes,>30,Emergency Room,Emergency,Discharged to home
8,12522,48330783,Caucasian,Female,[80-90),NaN,13,NaN,NaN,68,...,No,No,No,No,Ch,Yes,NO,Transfer from a hospital,Urgent,Discharged to home
9,15738,63555939,Caucasian,Female,[90-100),NaN,12,NaN,InternalMedicine,33,...,No,No,No,No,Ch,Yes,NO,Transfer from a hospital,Elective,Discharged/transferred to SNF


Ahora entendamos la información que tenemos en el _dataset_

In [32]:
aug_diabetic_df.set_index(["patient_nbr","encounter_id"], inplace = True)

## Data Wrangling

### Verifiquemos valores nulos de variables importantes

Antes de revisar valores nulos mal codificados, verifiquemos el total de valores nulos ya identificados

In [33]:
aug_diabetic_df.isna().sum()/aug_diabetic_df.shape[0]

race                        0.022336
gender                      0.000029
age                         0.000000
weight                      0.968585
time_in_hospital            0.000000
payer_code                  0.395574
medical_specialty           0.490822
num_lab_procedures          0.000000
num_procedures              0.000000
num_medications             0.000000
number_outpatient           0.000000
number_emergency            0.000000
number_inpatient            0.000000
diag_1                      0.000206
diag_2                      0.003518
diag_3                      0.013983
number_diagnoses            0.000000
max_glu_serum               0.947468
A1Cresult                   0.832773
metformin                   0.000000
repaglinide                 0.000000
nateglinide                 0.000000
chlorpropamide              0.000000
glimepiride                 0.000000
acetohexamide               0.000000
glipizide                   0.000000
glyburide                   0.000000
t

Hasta el momento tenemos algunos _features_ con hasta 10% de valores nulos

In [35]:
aug_diabetic_df.admin_src_desc .value_counts()

 Emergency Room                                               57494
 Physician Referral                                           29565
Transfer from a hospital                                       3187
 Transfer from another health care facility                    2264
Clinic Referral                                                1104
 Transfer from a Skilled Nursing Facility (SNF)                 855
HMO Referral                                                    187
 Not Mapped                                                     161
 Not Available                                                  125
 Court/Law Enforcement                                           16
 Transfer from hospital inpt/same fac reslt in a sep claim       12
 Transfer from critial access hospital                            8
Normal Delivery                                                   2
 Extramural Birth                                                 2
 Transfer from Ambulatory Surgery Center        

In [36]:
aug_diabetic_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 101766 entries, (8222157, 2278392) to (175429310, 443867222)
Data columns (total 48 columns):
race                        99493 non-null object
gender                      101763 non-null object
age                         101766 non-null object
weight                      3197 non-null object
time_in_hospital            101766 non-null int64
payer_code                  61510 non-null object
medical_specialty           51817 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            101766 non-null int64
diag_1                      101745 non-null object
diag_2                      101408 non-null object
diag_3                      100343 non-null object
number_diagnoses            101766 non-null int64
ma

### estudiar los outliers

In [38]:
aug_diabetic_df.columns

Index(['race', 'gender', 'age', 'weight', 'time_in_hospital', 'payer_code',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'admin_src_desc', 'admin_type_desc', 'dschrge_disp_desc'],
      dtype='object')